In [45]:
import pandas as pd

df = pd.read_csv(
    "data/OraclesElixir/2024_LoL_esports_match_data_from_OraclesElixir.csv",
    dtype={"url": "str"}
)

rows, cols = df.shape
print(f"The CSV file has {rows} rows and {cols} columns.")

print(len(df.columns.tolist()))
print(df.columns.tolist())


The CSV file has 117648 rows and 161 columns.
161
['gameid', 'datacompleteness', 'url', 'league', 'year', 'split', 'playoffs', 'date', 'game', 'patch', 'participantid', 'side', 'position', 'playername', 'playerid', 'teamname', 'teamid', 'champion', 'ban1', 'ban2', 'ban3', 'ban4', 'ban5', 'pick1', 'pick2', 'pick3', 'pick4', 'pick5', 'gamelength', 'result', 'kills', 'deaths', 'assists', 'teamkills', 'teamdeaths', 'doublekills', 'triplekills', 'quadrakills', 'pentakills', 'firstblood', 'firstbloodkill', 'firstbloodassist', 'firstbloodvictim', 'team kpm', 'ckpm', 'firstdragon', 'dragons', 'opp_dragons', 'elementaldrakes', 'opp_elementaldrakes', 'infernals', 'mountains', 'clouds', 'oceans', 'chemtechs', 'hextechs', 'dragons (type unknown)', 'elders', 'opp_elders', 'firstherald', 'heralds', 'opp_heralds', 'void_grubs', 'opp_void_grubs', 'firstbaron', 'barons', 'opp_barons', 'firsttower', 'towers', 'opp_towers', 'firstmidtower', 'firsttothreetowers', 'turretplates', 'opp_turretplates', 'inhib

In [42]:
for i, col in enumerate(df.columns):
    print(f"Column {i}: {col}")
    print(df[col].describe())
    if df[col].dtype == 'object' and df[col].nunique() <= 20:
        print("Unique values:", df[col].unique())
    print("------------------------------------")

Column 0: gameid
count               117648
unique                9804
top       LOLTMNT02_194401
freq                    12
Name: gameid, dtype: object
------------------------------------
Column 1: datacompleteness
count       117648
unique           2
top       complete
freq        100956
Name: datacompleteness, dtype: object
Unique values: ['partial' 'complete']
------------------------------------
Column 2: url
count                                            16692
unique                                             551
top       https://lpl.qq.com/es/stats.shtml?bmid=10934
freq                                                60
Name: url, dtype: object
------------------------------------
Column 3: league
count     117648
unique        51
top          LPL
freq        8604
Name: league, dtype: object
------------------------------------
Column 4: year
count    117648.000000
mean       2024.035292
std           0.194213
min        2023.000000
25%        2024.000000
50%        2024.00

In [ ]:
columns_to_drop = list(df.columns[1:11]) + ["playerid", 'teamname', 'teamid', 'playername', 'gamelength', 'infernals','' ]
df.drop(columns=columns_to_drop, axis=1, inplace=True)
filtered_rows = df[df['gameid'].isin(['LOLTMNT02_194401', 'LOLTMNT99_132665'])]
filtered_rows.to_csv("filtered_rows.csv", index=False)
filtered_rows




,gameid,side,position,playername,champion,ban1,ban2,ban3,ban4,ban5,...,opp_csat25,golddiffat25,xpdiffat25,csdiffat25,killsat25,assistsat25,deathsat25,opp_killsat25,opp_assistsat25,opp_deathsat25
192,LOLTMNT99_132665,Blue,top,Peop,Jax,Jarvan IV,Nocturne,Orianna,Cassiopeia,Darius,...,204.0,-2068.0,-638.0,-6.0,2.0,1.0,4.0,3.0,4.0,3.0
193,LOLTMNT99_132665,Blue,jng,Stalken,Karthus,Jarvan IV,Nocturne,Orianna,Cassiopeia,Darius,...,134.0,1868.0,3301.0,88.0,1.0,9.0,2.0,2.0,5.0,4.0
194,LOLTMNT99_132665,Blue,mid,alix,Lucian,Jarvan IV,Nocturne,Orianna,Cassiopeia,Darius,...,218.0,2421.0,1477.0,-11.0,7.0,6.0,2.0,1.0,1.0,3.0
195,LOLTMNT99_132665,Blue,bot,nevermind,Jhin,Jarvan IV,Nocturne,Orianna,Cassiopeia,Darius,...,227.0,-1087.0,-538.0,-31.0,6.0,4.0,0.0,5.0,1.0,3.0
196,LOLTMNT99_132665,Blue,sup,Noki,Braum,Jarvan IV,Nocturne,Orianna,Cassiopeia,Darius,...,42.0,325.0,-209.0,-15.0,1.0,8.0,3.0,0.0,4.0,4.0
197,LOLTMNT99_132665,Red,top,Sangrod,Jayce,Akshan,Neeko,Akali,Alistar,Ezreal,...,198.0,2068.0,638.0,6.0,3.0,4.0,3.0,2.0,1.0,4.0
198,LOLTMNT99_132665,Red,jng,Derakhil,Elise,Akshan,Neeko,Akali,Alistar,Ezreal,...,222.0,-1868.0,-3301.0,-88.0,2.0,5.0,4.0,1.0,9.0,2.0
199,LOLTMNT99_132665,Red,mid,sappxire1,Syndra,Akshan,Neeko,Akali,Alistar,Ezreal,...,207.0,-2421.0,-1477.0,11.0,1.0,1.0,3.0,7.0,6.0,2.0
200,LOLTMNT99_132665,Red,bot,tuokaZ,Varus,Akshan,Neeko,Akali,Alistar,Ezreal,...,196.0,1087.0,538.0,31.0,5.0,1.0,3.0,6.0,4.0,0.0
201,LOLTMNT99_132665,Red,sup,Rexha,Rell,Akshan,Neeko,Akali,Alistar,Ezreal,...,27.0,-325.0,209.0,15.0,0.0,4.0,4.0,1.0,8.0,3.0
